In [5]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

import pandapower as pp
import pandapower.networks
import pandapower.contingency
import pandapower.control
import pandapower.timeseries
import pandapower.plotting
from pandapower.pypower.makeYbus import makeYbus, branch_vectors
import numpy as np
from scipy.sparse import csr_matrix
from pandapower.pypower.idx_brch import *
from pandapower.pypower.idx_bus import BASE_KV, GS, BS

In [6]:
net = pp.networks.case24_ieee_rts()
pp.runopp(net)
net.res_line["from_bus"] = net.line.from_bus
net.res_line["to_bus"] = net.line.to_bus
net.res_trafo["hv_bus"] = net.trafo.hv_bus
net.res_trafo["lv_bus"] = net.trafo.lv_bus

ppc = net._ppc
branch = ppc["branch"]
bus = ppc["bus"]
nl = branch.shape[0]

Ytt, Yff, Yft, Ytf = branch_vectors(branch, nl)





In [7]:

V = net.res_bus.vm_pu * np.exp(1j * net.res_bus.va_degree * np.pi / 180)
Base_kv = bus[:, BASE_KV]
edge_index = np.real(branch[:, [F_BUS, T_BUS]]).astype(int)
rate_a = np.real(branch[:, RATE_A])
edge_attr = np.stack([Yff, Yft, Ytf, Ytt, rate_a], axis=1)

# shuffle edge_index and edge_attr by the same permutation
perm = np.random.permutation(edge_index.shape[0])
edge_index = edge_index[perm]
edge_attr = edge_attr[perm]

In [8]:
def check_assumptions(net):
    # check parallel = 1 
    assert np.all(net.line.parallel == 1)
    assert np.all(net.trafo.parallel == 1)


check_assumptions(net)


In [9]:
def get_line_index(from_bus, to_bus):
    # Return the index in edge_index where (from_bus, to_bus) matches
    idx = np.where((edge_index[:, 0] == from_bus) & (edge_index[:, 1] == to_bus))[0]
    if idx.size == 0:
        raise ValueError(f"Edge ({from_bus}, {to_bus}) not found in edge_index")
    return idx

In [10]:
def compute_branch_currents_kA(nb, nl, sn_mva, edge_index, edge_attr, V, Base_kv):
    """
    Computes from-end and to-end branch currents in kA.

    Parameters:
    - nb: int, number of buses
    - nl: int, number of branches (lines)
    - sn_mva: float, system base power in MVA
    - edge_index: np.ndarray of shape (nl, 2), with [from_bus, to_bus] per branch
    - edge_attr: np.ndarray of shape (nl, 4), with [Yff, Yft, Ytf, Ytt] per branch
    - V: np.ndarray of complex bus voltages in per-unit (shape: nb,)
    - Base_kv: np.ndarray of shape (nb,), base voltage in kV

    Returns:
    - If_kA: np.ndarray of shape (nl,), from-end current magnitudes in kA
    - It_kA: np.ndarray of shape (nl,), to-end current magnitudes in kA
    """
    f = edge_index[:, 0]
    t = edge_index[:, 1]

    Yff = edge_attr[:, 0]
    Yft = edge_attr[:, 1]
    Ytf = edge_attr[:, 2]
    Ytt = edge_attr[:, 3]

    Vf_base_kV = Base_kv[f]
    Vt_base_kV = Base_kv[t]

    i = np.hstack([np.arange(nl), np.arange(nl)])

    # From-end admittance matrix and current
    Yf = csr_matrix((np.hstack([Yff, Yft]), (i, np.hstack([f, t]))), shape=(nl, nb))
    If_pu = Yf @ V
    If_kA = np.abs(If_pu) * sn_mva / (np.sqrt(3) * Vf_base_kV)

    # To-end admittance matrix and current
    Yt = csr_matrix((np.hstack([Ytf, Ytt]), (i, np.hstack([f, t]))), shape=(nl, nb))
    It_pu = Yt @ V
    It_kA = np.abs(It_pu) * sn_mva / (np.sqrt(3) * Vt_base_kV)

    return If_kA, It_kA


def check_branch_currents(net, edge_index, If_kA, It_kA, decimals=6):
    """
    Compares calculated branch currents and loading with pandapower results.

    Parameters:
    - net: pandapower network
    - edge_index: np.ndarray of shape (n_edges, 2), from and to bus indices
    - If_kA: np.ndarray of calculated from-end branch currents
    - It_kA: np.ndarray of calculated to-end branch currents
    - decimals: int, number of decimal places to round to
    """
    
    # create sets of all (from_bus, to_bus, I_from_kA, I_to_kA) for lines
    set_of_lines = set((line.from_bus, line.to_bus, np.round(line.i_from_ka, decimals), np.round(line.i_to_ka, decimals)) for line in net.res_line.itertuples())
    # add trafos to the set
    set_of_lines.update((trafo.hv_bus, trafo.lv_bus, np.round(trafo.i_hv_ka, decimals), np.round(trafo.i_lv_ka, decimals)) for trafo in net.res_trafo.itertuples())
    set_of_lines_computed = set((edge_index[line_index, 0], edge_index[line_index, 1], np.round(If_kA[line_index], decimals), np.round(It_kA[line_index], decimals)) for line_index in range(edge_index.shape[0]))
    # assert that the two sets are equal
    assert set_of_lines == set_of_lines_computed, "Lines do not match"






In [11]:
If_kA, It_kA = compute_branch_currents_kA(bus.shape[0], branch.shape[0], net.sn_mva, edge_index, edge_attr, V, Base_kv)

In [12]:
# check the results
check_branch_currents(net, edge_index, If_kA, It_kA, decimals=6)


In [13]:
def compute_loading(edge_index, If_kA, It_kA, base_kv, edge_attr):
    """
    Compute per-branch loading using current magnitudes and branch ratings.

    Parameters:
    - edge_index: np.ndarray of shape (n_edges, 2), each row is [from_bus, to_bus]
    - If_kA: np.ndarray of from-side current magnitudes in kA
    - It_kA: np.ndarray of to-side current magnitudes in kA
    - base_kv: np.ndarray of shape (n_buses,), base voltage in kV per bus
    - edge_attr: np.ndarray of shape (n_edges, >=5), edge features, column 4 = RATE_A

    Returns:
    - loading: np.ndarray of shape (n_edges,), max of from and to side loading
    """
    from_bus = edge_index[:, 0]
    to_bus = edge_index[:, 1]
    Vf_base_kV = base_kv[from_bus]
    Vt_base_kV = base_kv[to_bus]

    rateA = np.real(edge_attr[:, 4])

    limitf = rateA / (Vf_base_kV * np.sqrt(3))
    limitt = rateA / (Vt_base_kV * np.sqrt(3))

    loadingf = If_kA / limitf
    loadingt = It_kA / limitt

    return np.maximum(loadingf, loadingt)


In [14]:
loading = compute_loading(edge_index, If_kA, It_kA, Base_kv, edge_attr)


In [15]:
def check_loading(net, edge_index, loading, decimals=6):
    """
    Compare computed loading against values in net.res_line and net.res_trafo.

    Parameters:
    - net: pandapower network
    - edge_index: np.ndarray of shape (n_edges, 2), mapping lines/transformers to buses
    - loading: np.ndarray of computed loading (shape = number of branches)
    """
    
    # create sets of all (from_bus, to_bus, loading) for lines
    set_of_lines = set((line.from_bus, line.to_bus, np.round(line.loading_percent / 100, decimals)) for line in net.res_line.itertuples())
    # add trafos to the set
    set_of_lines.update((trafo.hv_bus, trafo.lv_bus, np.round(trafo.loading_percent / 100, decimals)) for trafo in net.res_trafo.itertuples())
    set_of_lines_computed = set((edge_index[line_index, 0], edge_index[line_index, 1], np.round(loading[line_index], decimals)) for line_index in range(edge_index.shape[0]))
    # assert that the two sets are equal
    assert set_of_lines == set_of_lines_computed, "Lines do not match"
    


In [16]:
check_loading(net, edge_index, loading)